In [27]:
import numpy as np

# Function to preprocess the given text files
# Returns X, which contains the edges of each input graph
# Returns Y, which contains connectedness of each input graph (1 or 0)
def preprocess(filename):
    #Read all lines from txt file
    file = open(filename + '.txt', "r")
    lines = file.readlines()
    file.close
    
    lines[:] = [line for line in lines if line[0] == '1' or line[0] == '0']
    
    numInput = len(lines)
    numEdges = len(lines[0].split()) - 1
    
    #each row in X is a graph, each column is an edge
    #Y stores Y/N connected. 1 = connected, 0 = unconnected
    X = np.ones(shape=(numInput, numEdges))
    Y = np.ones(shape=(numInput, 1))
    
    for i, line in enumerate(lines):
        inputs = line.split()
        if inputs[-1] == 'Y':
            Y[i] = 1
        else:
            Y[i] = 0
            
        for j, edge in enumerate(inputs[0:numEdges]):
            X[i][j] = int(edge)
    
    return X, Y

X, y = preprocess(filename="graphs5")

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable

In [23]:
class My_Neural_Net(nn.Module):    
    #constructor
    #take in X as a parameter
    def __init__(self, X):
        super(My_Neural_Net, self).__init__()
        
        #Find dimensionality of X
        X_dim = X.shape[1]
        
        # Define the layers.
        numFeatures=20
        self.layer_1 = nn.Linear(X_dim, numFeatures)
        self.layer_2 = nn.Linear(numFeatures,numFeatures)
        self.layer_3 = nn.Linear(numFeatures, numFeatures)
        self.layer_4 = nn.Linear(numFeatures, numFeatures)
        self.layer_5 = nn.Linear(numFeatures, 1)

        # Define activation functions.  
        # sigmoid.
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
        # Define what optimization we want to use.
        # Adam is a popular method so I'll use it.
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.0001)
        
    # 1. input X
    def forward(self, X):
        # 2. linearly transform X into hidden data 1
        X = self.layer_1(X)
        # 3. perform ReLU on hidden data
        X = self.relu(X)
        # 4. linearly transform hidden data into hidden data 2
        X = self.layer_2(X)
        # 5. perform ReLU on hidden data
        X = self.relu(X)
        # 6. linearly transform hidden data into output layer
        X = self.layer_3(X)
        
        X = self.relu(X) 
        X = self.layer_4(X)
        X = self.relu(X)
        X = self.layer_5(X)
        
        # 7. perform sigmoid on output data to get f(X) predictions between 0 and 1
        X = self.sigmoid(X)
        
        # 8. output predictions
        return X
    
    def loss(self, pred, true):
        #PyTorch's own cross entropy loss function.
        score = nn.BCELoss()
        return score(pred, true)
    

    # 1. input: N - number of iterations to train, X - data, y - target
    def fit(self,X,y,N = 5000):
        
        # 2. for n going from 0 to N -1 :
        for epoch in range(N):
            
            # Reset weights in case they are set for some reason
            self.optimizer.zero_grad()
            
            # 3. f(X) = forward(X) 
            pred = self.forward(X)
            
            # 4. l = loss(f(X),y)
            l = self.loss(pred, y)
            #print loss
            print(l)
            
            # 5. Back progation
            l.backward()
            # 5. Gradient Descent
            self.optimizer.step()
    
    def predict_proba(self, X):
        # probability of being a 1
        prob_1 = self.forward(X)
              
        # vectorwise subtraction
        prob_0 = 1 - prob_1
        
        # make into a matrix
        probs = torch.cat((prob_0,prob_1), dim = 1)
        
        return probs
    
    def predict(self, X):
        probs = self.predict_proba(X)
        
        # get only second column (probability of being a 1)
        probs_1 = probs[:,1:]
        
        # 1 if prob_1 is greater or equal to than 0.5 for a given example
        # 0 if less than 0.5
        preds = (probs_1 >= 0.5).int()
        
        return preds
    
    def score(self, X, y):
        # proportion of times where we're correct
        # conversions just allow the math to work
        acc = (self.predict(X) == y.int()).float().mean()
        
        return acc

In [28]:
# Split into train and test so we can fit on some data and see performance 
# on data we havent seen yet.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

# Turn X and y (train and test) into PyTorch objects. We always have to do this step
X_train_tens = Variable(torch.Tensor(X_train).float())
X_test_tens = Variable(torch.Tensor(X_test).float())
y_train_tens = Variable(torch.Tensor(y_train).float())
y_test_tens = Variable(torch.Tensor(y_test).float())

neur_net = My_Neural_Net(X_train_tens)
neur_net.fit(X_train_tens,y_train_tens)
neur_net.score(X_test_tens,y_test_tens)

tensor(0.7158, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7156, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7154, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7152, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7150, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7148, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7146, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7145, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7143, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7141, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7139, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7137, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7135, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7133, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7131, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7129, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7127, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7126, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.7124, grad_fn=<BinaryCrossEntropyBack

tensor(0.6830, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6828, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6826, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6824, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6823, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6821, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6819, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6817, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6815, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6814, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6812, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6810, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6808, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6806, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6805, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6803, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6801, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6799, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6797, grad_fn=<BinaryCrossEntropyBack

tensor(0.6365, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6359, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6353, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6347, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6341, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6335, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6328, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6322, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6316, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6310, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6304, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6297, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6291, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6285, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6278, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6272, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6266, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6259, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.6253, grad_fn=<BinaryCrossEntropyBack

tensor(0.5389, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5384, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5379, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5374, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5369, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5364, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5359, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5355, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5350, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5345, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5340, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5335, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5330, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5325, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5320, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5316, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5311, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5306, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.5301, grad_fn=<BinaryCrossEntropyBack

tensor(0.4462, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4456, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4451, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4445, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4439, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4433, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4428, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4422, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4416, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4411, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4405, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4399, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4393, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4388, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4382, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4376, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4371, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4365, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.4359, grad_fn=<BinaryCrossEntropyBack

...

tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0028, grad_fn=<BinaryCrossEntropyBack

tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0022, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0021, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0021, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0021, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0021, grad_fn=<BinaryCrossEntropyBack

tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0017, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0016, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0016, grad_fn=<BinaryCrossEntropyBack

tensor(0.0014, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0014, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBackward>)
tensor(0.0013, grad_fn=<BinaryCrossEntropyBack

tensor(0.9756)